In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
pip install youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 10.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

def extract_video_id(url: str) -> str:
    """
    Extract the YouTube video ID from a URL.
    Raises ValueError if no 'v' parameter is found.
    """
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    video_ids = qs.get('v')

    if not video_ids:
        raise ValueError(f"No video id found in URL: {url}")
    return video_ids[0]

if __name__ == "__main__":
    url = "https://www.youtube.com/watch?v=iG9CE55wbtY&list=PL68492A0FC99185C7"

    video_id = extract_video_id(url)

    api = YouTubeTranscriptApi()
    fetched = api.fetch(video_id, languages=['en']) # returns a FetchedTranscript

    text = "\n".join(snippet.text for snippet in fetched)

In [3]:
def chunk_text(text, max_chars = 3500, chunk_size = 200):
    """
    Because facebook/bart-large-cnn model is limited to 1024 tokens,
    we will chunk the text into 200 tokens for each chunk
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_chars
        chunk = text[start:end]
        chunks.append(chunk)
        if (end >= len(text)):
            break
        start = end + 1
    return chunks

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

2026-01-06 19:35:05.543607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767728105.750957      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767728105.808789      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767728106.313204      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767728106.313237      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767728106.313240      55 computation_placer.cc:177] computation placer alr

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [5]:
#print(pipe(text, max_length=130, min_length=30, do_sample=True))
chunks = chunk_text(text)
summaries = []
for i, chunk in enumerate(chunks):
    print(f"Summarizing chunk {i+1} of {len(chunks)}")
    summary = pipe(chunk, max_length=130, min_length=30, do_sample=True)[0]["summary_text"]
    summaries.append(summary)

Summarizing chunk 1 of 6
Summarizing chunk 2 of 6
Summarizing chunk 3 of 6
Summarizing chunk 4 of 6
Summarizing chunk 5 of 6


Your max_length is set to 130, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


Summarizing chunk 6 of 6


In [6]:
combined_summary = " ".join(summaries)
print("Final Combined Summary:\n")
print(combined_summary)

Final Combined Summary:

Imitation is as important in education as literacy as literacy is in literacy, he says. He says kids have tremendous talents, and we squander them, pretty ruthlessly. "I heard a great story recently -- I love telling it -- of a little girl who was in a drawing lesson," he adds. James Robinson: Kids will take a chance if they don't know, they'll have a go. By the time they get to be adults, most kids have lost that capacity, he says. Robinson: We run our companies like this. We stigmatize mistakes. Every education system on earth has the same hierarchy of subjects. Maths and languages are at the top, the humanities at the bottom. Art and music are normally given a higher status in schools than drama and dance. In the next 30 years, according to UNESCO, more people worldwide will be graduating through education than at any point in history. David Frum: We need to radically rethink our view of intelligence. He says creativity often comes about through interaction 